# Dynamic wakes 1

## Spatially homogeneous wake propagation

For spatially homogeneous timeseries input data, `foxes` can compute dynamic wake propagation. This in principle works by following a flow trace backwards in time for each point of interest, and identifying it with a wake trajectory if it hits a rotor.

Since all `foxes` computations are based on _chunks_ of input states, this concept only works if

- either all states fall into a single chunk,
- or the `Iterative` algorithm is used for the calculation.

The later is necessary in case the wake originates from a state previous to the chunk of evaluation, since the default `Downwind` algorithm does not allow cross-chunk communication during the calculation. 

These are the inlcudes for this example:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["animation.html"] = "jshtml"

import foxes
import foxes.variables as FV
import foxes.constants as FC

We create a case with a regular 3 x 3 wind farm layout:

In [ ]:
states = foxes.input.states.Timeseries(
    data_source="timeseries_100.csv.gz",
    output_vars=[FV.WS, FV.WD, FV.TI, FV.RHO],
    var2col={FV.WS: "ws", FV.WD: "wd", FV.TI: "ti"},
    fixed_vars={FV.RHO: 1.225, FV.TI: 0.07},
)

farm = foxes.WindFarm()
foxes.input.farm_layout.add_grid(
    farm,
    xy_base=np.array([0.0, 0.0]),
    step_vectors=np.array([[1000.0, 0], [0, 800.0]]),
    steps=(3, 3),
    turbine_models=["DTU10MW"],
    verbosity=0
)

algo = foxes.algorithms.Iterative(
    farm,
    states,
    rotor_model="grid25",
    wake_models=["Bastankhah2014_linear_k004"],
    wake_frame="timelines",
    partial_wakes="rotor_points",
    chunks={FC.STATE: 500, FC.POINT: 5000},
    verbosity=1
)

Notice the wake frame choice `timelines`, which is a pre-defined instance of the class `Timelines` from the model book.

Let's run the wind farm calculation:

In [ ]:
with foxes.utils.runners.DaskRunner() as runner:
    farm_results = runner.run(algo.calc_farm)

Notice the iterations and the convergence behaviour. Now the farm results are ready:

In [ ]:
farm_df = farm_results.to_dataframe()
print("\nFarm results data:\n")
print(farm_df[[FV.AMB_REWS, FV.REWS, FV.P]])

This timeseries has a time step of 1 minute. Let's visualize the wake dynamics in am animation:

In [ ]:
with foxes.utils.runners.DaskRunner() as runner:

    fig, axs = plt.subplots(2, 1, figsize=(5.2,7), 
                            gridspec_kw={'height_ratios': [3, 1]})

    anim = foxes.output.Animator(fig)

    # this adds the flow anomation to the upper panel:
    of = foxes.output.FlowPlots2D(algo, farm_results, runner=runner)
    anim.add_generator(
        of.gen_states_fig_xy(
            FV.WS,
            resolution=30,
            quiver_pars=dict(scale=0.013),
            quiver_n=35,
            xmax=5000,
            ymax=5000,
            fig=fig,
            ax=axs[0],
            vmin=0,
            vmax=6,
            title=None,
            ret_im=True,
            animated=True,
        )
    )

    # This adds the REWS signal animation to the lower panel:
    o = foxes.output.FarmResultsEval(farm_results)
    anim.add_generator(
        o.gen_stdata(
            turbines=[4, 7],
            variable=FV.REWS,
            fig=fig,
            ax=axs[1],
            ret_im=True,
            legloc="upper left",
            animated=True,
        )
    )

    # This adds turbine indices at turbine positions:
    lo = foxes.output.FarmLayoutOutput(farm)
    lo.get_figure(
        fig=fig,
        ax=axs[0],
        title="",
        annotate=1,
        anno_delx=-120,
        anno_dely=-60,
        alpha=0,
    )
    
    ani = anim.animate()
    plt.close()
    print("done.")

print("Creating animation")
ani

For the fun of it, let's re-run this case assuming the time step was 10 s instead of 1 min. We can do so by using the wake frame `Timelines(dt_min=1/6)`, which is called `timelines_10s` in the model book:

In [ ]:
algo.finalize(clear_mem=True)

algo = foxes.algorithms.Iterative(
    farm,
    states,
    rotor_model="grid25",
    wake_models=["Bastankhah2014_linear_k004"],
    wake_frame="timelines_10s",
    partial_wakes="rotor_points",
    chunks={FC.STATE: 500, FC.POINT: 5000},
    verbosity=1
)

with foxes.utils.runners.DaskRunner() as runner:
    
    farm_results = runner.run(algo.calc_farm)

    fig, axs = plt.subplots(2, 1, figsize=(5.2,7), 
                            gridspec_kw={'height_ratios': [3, 1]})

    anim = foxes.output.Animator(fig)

    # this adds the flow anomation to the upper panel:
    of = foxes.output.FlowPlots2D(algo, farm_results, runner=runner)
    anim.add_generator(
        of.gen_states_fig_xy(
            FV.WS,
            resolution=30,
            quiver_pars=dict(scale=0.013),
            quiver_n=35,
            xmax=5000,
            ymax=5000,
            fig=fig,
            ax=axs[0],
            vmin=0,
            vmax=6,
            title=lambda si, s: f"t = {si/6:3.2f} min",
            ret_im=True,
            animated=True,
        )
    )

    # This adds the REWS signal animation to the lower panel:
    o = foxes.output.FarmResultsEval(farm_results)
    anim.add_generator(
        o.gen_stdata(
            turbines=[4, 7],
            variable=FV.REWS,
            fig=fig,
            ax=axs[1],
            ret_im=True,
            legloc="upper left",
            animated=True,
        )
    )

    # This adds turbine indices at turbine positions:
    lo = foxes.output.FarmLayoutOutput(farm)
    lo.get_figure(
        fig=fig,
        ax=axs[0],
        title="",
        annotate=1,
        anno_delx=-120,
        anno_dely=-60,
        alpha=0,
    )
    
    ani = anim.animate()
    plt.close()
    print("done.")

print("Creating animation")
ani